# Phys 260: Python assignment header

### (1) Fill out the cell below.  
The cell below is a **code cell**.  Fill out your University of Michigan uniqname, then your name, and collaborators in the cell below **inside the quotes**.  

**Do not delete the quotes.**  We will use this information to organize your assignments.  To edit and execute cells, double click inside the cell, type, and press \<shift\>+\<enter\> to execute.

In [ ]:
UNIQNAME = ""
NAME = ""
COLLABORATORS = ""

### (2) Check your python version.  
**Execute the cell below** (double click in the cell and press \<shift\>+\<enter\>, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

### (3) Do your best to answer all questions in the assignment.  
To answer questions, **replace** anything that says either
- "YOUR ANSWER HERE" 
- 
```
YOUR CODE HERE
raise NotImplementedError
``` 

with your answer/code.  Cells with either of the two bullet points above are cells of the notebook that will be graded.

**To edit markdown** cells (e.g. this one),  *double click in the cell to type*.  Press \<shift\>+\<enter\> to execute the cell.  Try editing the text below to replace the with your information (e.g. Camille Avestruz, cavestru):  

[first name] [last name], uniqname


### (4) Make sure your notebook runs sequentially.
After you complete this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Introduction -- Reminder

Each Python lab will start with a pre-flight exercise that walks through building some of the set up and tools ($\sim$ 30 min), followed by an in-class tutorial with time for Q+A (50 min) so you can walk through steps that will be necessary for the homework assignment you will submit ($\sim$ 3 hrs).  Each lab will contain starter code, similar to what you see below.  Please fill in the code to complete the pre-flight assignment in preparation for the in-class tutorial.  

Preflight ($\sim$30-60 min, 10 points) **Typically due: Wednesdays 3pm EST**

*Preflight typically graded by Wednesday 5p EST*

In-class tutorial and Q+A ($\sim$ 50 min, 10 points) **Typically occurs: Thursdays 9am EST**

Homework assignment ($\sim$ 3-5 hrs, 30 points) **Typically due: Mondays 9am EST** 

*Homework typically graded by Thursday 5p*

When we grade your homework, we will not run your code. Once submitted, your notebook should have the outputs for all of your results.  Please do not include long outputs from debugging, beyond a few print statements and the requested visualizations (i.e. plots).

**Grading:** When we grade your notebook, we will convert the .ipynb file to an HTML file.  We will be using [nbgrader](https://nbgrader.readthedocs.io/en/stable/) to grade your notebooks.  **Note:** If your ```IPython``` version is too old, we will *not* be able to grade your assignments.


# Phys 260 Python Lab 3: Boundary conditions (15 points total)

## Tutorial summary
- Chat over the background
- Method of relaxation for setting the potential with boundary conditions (1-d version)
- Plotting a scalar field (e.g. the potential V)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

## Background of application (quick review of the vector calculus between $\vec{E}$ and V)

In lectures you saw, 
\begin{equation}
\vec E = - \vec \nabla V, ~~~~~~~~ ~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (1) }
\end{equation}
where $\vec \nabla = \left(\frac{\partial}{\partial x},\frac{\partial}{\partial y},\frac{\partial}{\partial z}\right)$ in cartesian coordinates,
 and 
 \begin{equation}
\vec E = \int \frac {\rho d\tau} {4 \pi \epsilon_0}\frac {\hat r} {r^2}.~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (2) }
\end{equation}
Here $d\tau$ is the differential volume element (e.g. $d\tau=dxdydz$) to avoid confusion with $V$ which is the potential.

Equation (2) is an integral equation.  The inverse of integration is taking a derivative.  Taking the divergence of both sides of Equation (2), we arrive at the differential form of Gauss' law:

\begin{equation}
\vec \nabla \cdot \vec E = \frac \rho {\epsilon_0}. ~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (3) }
\end{equation}

If we now substitute Equation (1) into this equation we get: 
\begin{equation}
\nabla^2 V= \frac {- \rho} {\epsilon_0}.~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (4) }
\end{equation}

It might help to write out how this works out in Cartesian coordinates in case the nabla-notation is new to you:

\begin{equation}
\vec{\nabla}_{\mathrm{cart}}=\left[\frac{\delta}{\delta x},\frac{\delta}{\delta y}, \frac{\delta}{\delta z}\right]
=\left[\delta_x,\delta_y,\delta_z\right] = \delta_x\hat{i} + \delta_y\hat{j} + \delta_z\hat{k}
\end{equation}

Expressing the electric field in terms of the potential, we have,
\begin{equation}
\vec{E} = -\vec{\nabla}_{\mathrm{cart}}V=-\left[\frac{\delta V}{\delta x},\frac{\delta V}{\delta y}, \frac{\delta V}{\delta z}\right]
=-\left[\delta_xV,\delta_yV,\delta_zV\right] = -\delta_xV\hat{i} - \delta_yV\hat{j} - \delta_zV\hat{k} = E_x\hat{i}+E_y\hat{j}+E_z\hat{k}
\end{equation}

And, explicitly writing out equation (3):
\begin{equation}
\vec \nabla \cdot \vec E = \delta_xE_x+\delta_yE_y+\delta_zE_z = \frac \rho {\epsilon_0}. 
\end{equation}

And, explicitly writing out equation (4) in cartesian coordinates:
\begin{equation}
\left(\vec \nabla \cdot (-\vec{\nabla}V\right) = \left[\delta_x,\delta_y,\delta_z\right] \cdot \left[-\delta_xV,-\delta_yV,-\delta_zV\right] = -\left(\delta_x^2V + \delta_y^2V + \delta_z^2V\right) = -\nabla^2 V,
\end{equation}
so, 
\begin{equation}
\nabla^2V = -\frac{\rho}{\epsilon_0}
\end{equation}
where, $\nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}$.

This simplifies when we restrict ourselves to the case where we are considering the potential in regions where there are absolutely no charges (e.g., $\rho = 0$).  Then we have:
\begin{equation}
\nabla^2 V= 0.  ~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (5) }
\end{equation}
Equation 5 is just saying that the second derivative of the potential $V$ is zero in regions where there are no charges.  If this were one dimensional then this would mean that the potential can have slopes and nothing more complicated that that.    

A TLDR: The ingredients for Equation 5:
- Coulomb's law, Equation (2) and (3) with the integral and differential forms
- The definition of the potential, Equation (1) 
- The choice that we will only consider regions that are charge free.  

Assuming our problem satisfies condition (c) then Equation 5 completely specifies the electric field and is equivalent to solving Coulomb's law.  Of course you need Equation (1) to recover the field.   You might think that (c) is a big restriction, but it isn't.  You just carefully cut out the regions that have charges, figure out the potential on the boundary of the regions you had to remove and then plug ahead with (5).   


The generalization of this statement to 2 and 3 dimensions will lend itself to a very easy numerical method for solving for the potential.  We'll come back to this in the next section.

## Relaxing the potential in 1-d

Now, we exploit the fact that "the potential has slopes, and nothing else" for the $\rho=0$ (i.e. $\nabla^2V=0$), and we use the method of relaxation to calculate the potential at all points, given the boundary conditions.  

For 1-d, (e.g. $\frac{d^2 V(x)}{d x^2} = 0$), the solution is $V(x) = ax + b$, or a straight line with a slope.  For a straight line, the potential at any point is the average of the value at points an equal distance to the right and left of that point, e.g. $V(x_i) = \left(V(x_{i+1}) + V(x_{i-1})\right)/2$.  

The extension to two dimensions is that the potential at any point is the average of the potential in a circle around that point (all points equidistant in 2d).  The extension to three dimensions is that potential at any point is the average of the potential in a sphere (again, all points equidistant in 3d).  We are exploiting [Taylor's theorem](https://en.wikipedia.org/wiki/Taylor%27s_theorem).  

**Group exercise** (5 min -- 2 points): Let us first set up boundary conditions of the potential in 1-d.  
- First, set up some one-dimensional position coordinates, `x_positions_1d = np.arange(0,20,0.2)`
- Start with a 1-d array, `potential_1d` of zeros with the same shape as `x_positions_1d`.
- Set the boundary conditions for the first (left edge) and last elements (right edge) of `potential_1d` to be -100 and 100 respectively

In [ ]:
# Define x_positions_1d, and potential_1d here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(potential_1d.shape == x_positions_1d.shape)

Now, we numerically enforce that $\frac{d^2 V(x)}{d x^2} = 0$, by iteratively setting $V(x_i) = \left(V(x_{i+1}) + V(x_{i-1})\right)/2$ and setting the boundary conditions.  Note, this is now the `potential_right` and `potential_left` that we can define using `np.roll` from the preflight.  In the next exercise, we do *one* such iteration.

**Group exercise** (5 min -- 2 points): Define a function `relax_potential_1d` that takes in the potential array as an argument, uses `np.roll` to define `potential_right` ($V(x_{i+1})$) and `potential_left` ($V(x_{i-1})$), and returns the average with the boundary conditions set.  Below, you'll need to: 
- fill in the missing pieces of the function by defining `potential_right` and 
- enforce the boundary conditions determined by the key word arguments as you did in the previous exercise.

In [ ]:
def relax_potential_1d(potential_array, left_edge_condition = -100, right_edge_condition = 100) :
    """Returns relaxed 1-d potential array with set boundary conditions.  

    Inputs:
    potential_array (n-darray) : array containing the potential at each sampled point 
    left_edge_condition (float) : value of potential on left edge
    right_edge_condition (float) : value of potential on right edge

    Outputs:
    relaxed_potential (n-darray) : array containing the relaxed potential at each sampled point with boundary conditions set     
    """
    
    # Define potential_left and potential_right using np.roll (i.e. V(x_{i-1}) and V(x_{i+1})
    potential_left = np.roll(potential_array,-1) # This is V(x_{i-1})
    # YOUR CODE HERE
    raise NotImplementedError()
    
    relaxed_potential = np.mean(np.array([potential_left, potential_right]),axis=0)
    
    # Set boundary conditions
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return relaxed_potential

In [ ]:
"""Execute to check you're on the right track"""
assert(relax_potential_1d(potential_1d).shape == potential_1d.shape)

### Iterate over the relaxation step

Let's now iterate over relaxing the potential for 500 steps, and see how it changes every 100 steps.

In [ ]:
# Initialize the potential
potential_1d = np.zeros(x_positions_1d.shape)

# Take snapshots of the relaxing potential
potential_snapshots = []

# Relax over 500 iterations
for step in np.arange(500) :
    potential_1d = relax_potential_1d(potential_1d)
    if step % 100 == 0 :
        potential_snapshots.append(potential_1d)

### Plot snapshots of the relaxation

In [ ]:
for i, potential_snapshot in enumerate(potential_snapshots) :
    plt.plot(x_positions_1d, potential_snapshot, label='%ith step'%(i*100))
    
plt.xlabel('x', fontsize='xx-large')
plt.ylabel('V', fontsize='xx-large')
plt.legend()

**Discussion question** (2 min -- 2 points) : What is happening to the potential as we do more and more iterations (steps)?  What do you expect to happen as we go towards infinite iterations (e.g. steps $\rightarrow\inf$)?

YOUR ANSWER HERE

**Group Exercise:** (10 min -- 5 points)  You will now refactor the code where we relaxed the potential over 500 steps.  *Code refactoring* refers to improving existing code.

Old code:
```
# Initialize the potential
potential_1d = np.zeros(x_positions_1d.shape)

# Take snapshots of the relaxing potential
potential_snapshots = []

# Relax over 500 iterations
for step in np.arange(500) :
    potential_1d = relax_potential_1d(potential_1d)
    if step % 100 == 0 :
        potential_snapshots.append(potential_1d)
```
In the next cell, you will refactor this code to continue iterations until "convergence" to avoid hard coding the number of iterations.  In this case, your code will not assume a number of iterations (e.g. 500), but stops once the largest difference between the potential at a given point in the previous iteration and the potential at that given point in the current iteration is less than 0.01.  We will use 0.01 as a criterion for *convergence*, and you will use a `while` loop.  

A simple example of the `while` loop is:
```
i = 20
while i > 10 : 
    i -= 1
    print(i)
```
The above code will print 20, 19, ... 11 and then exit the loop once the boolean fails.

The skeleton to the code you will need to write is below where we have already defined `convergence_criterion`, initialized the `largest_iteration_difference` (serving the same purpose as the variable i in the example).  The step of initializing the potential, and creating an empty `list`, `largest_iteration_differences` to collect the largest iteration is also already below.  You will need to fill out the `while` loop. Some suggestions:
- You will need to compare both the relaxed potential (i.e. the returned array from the function `relax_potential_1d`) and the potential just before the relaxation.  The first line inside the `while` loop will help you do this.
- You will care about the largest difference between the two, e.g. `np.max(np.abs(relaxed_potential - potential_1d))`.  

In [ ]:
# Refactored code here

convergence_criterion = 0.01
largest_iteration_difference = 1 # initialize

# Initialize the potential
potential_1d = np.zeros(x_positions_1d.shape)

# Collect the largest_iteration_difference
largest_iteration_differences = []

while largest_iteration_difference > convergence_criterion :
    relaxed_potential = relax_potential_1d(potential_1d)
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""Execute to check you are on the right track"""
assert(largest_iteration_difference <= convergence_criterion)

Below, we plot 
- how quickly the relaxation method converged, and also 
- the final 1d potential.

Note, we make use of the subplots method, where we can have multiple axes on a single figure.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,7))
ax1.plot(largest_iteration_differences)
ax1.set_yscale('log')
ax1.set_xlabel('Iteration', fontsize='xx-large')
ax1.set_ylabel('$(\Delta V)_{max}$', fontsize='xx-large')

ax2.plot(x_positions_1d, potential_1d)
ax2.set_xlabel('x', fontsize='xx-large')
ax2.set_ylabel('V', fontsize='xx-large')

## Calculate the electric field

Now, we will take the 1-d potential, and calculate the electric field.  Recall, to calculate $E$ from $V$, you must calculate the vector of partial derivatives of $V$. In 1d, to compute $E_x = -\partial V/\partial x$, you would take a first derivative using only the $x$-components of $V$.  

**Quick exercise** (2 min -- 2 points) : Define a function `calc_efield_1d(potential_array)` using `np.gradient`.

In [ ]:
def calc_efield_1d(potential_array) :
    """Calculate the electric field in one dimension

    Inputs:
    potential_array (n-darray) : array containing the potential at each sampled point 

    Outputs:
    efield_array (n-darray) : array containing the corresponding efield at each sampled point     
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert((calc_efield_1d(np.zeros(20))==np.zeros(20)).all())

**Quick plot** (2 min -- 2 points): Use the `plt.subplots` method to generate two axes on a single figure on which you will plot the 1d potential and 1d electric field side by side.  You'll notice some noise in the electric field.  

Try setting `convergence_criterion = 0.001` where it was set before, and run all cells after.  How does the electric field change? 

In [ ]:
# Plot the potential and efield here
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,7))
# YOUR CODE HERE
raise NotImplementedError()

# Phys 260 Python Lab 3: Relaxing the potential in 2d with boundary conditions: The setup for modeling a capacitor (15 points total)


Your homework assignment begins here.  Note, the homework below relies on code built during the tutorial (above)


## Homework summary 
- Extending the method of relaxation in 2d
- Plotting the 2d scalar field of the potential and vector field of $\vec{E}$

The extension to 2d sets us up for modeling a capacitor, which we will do next week.  

Our 2-d potential that we will discretize and solve for will be Equation (5), explicitly written as:
\begin{equation}
\frac{\partial^2V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}  = 0.~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (6) }
\end{equation}



In 2d, the relaxation process involves setting the potential at any given point equal to the potential in a circle around it.  On a grid, this corresponds to the points above, below, to the left, and to the right. 

You will do an analogous iterative method to solve for the potential in 2d. Recall the steps:

Step 1:  Use `np.meshgrid` to divide the space into a grid with a uniform grid spacing $\delta$.

Step 2:  Initialize the potential of the grid to match the boundary conditions specified in the problem.

Step 3:  Replace the potential within each cell inside the grid with the average of the potential in the cells above, below, left and right of it, enforcing boundary conditions.

\begin{equation}
V(x,y) = \frac 1 4 \left[ V(x+\delta,y) + V(x-\delta,y) + V(x,y+\delta) + V(x,y-\delta)\right]~~~~~~~~ ~~~~~~~~ ~~~~~~~~ \mbox{  (7) }
\end{equation}

Step 4: Repeat steps 2 and 3 enough times for the results to converge.  Use the new grid as input for step 3 each time you repeat.

To do this, we write other functions that:

- Sets the boundary conditions in 2d with `set_boundary_conditions`
- Relax the potential through an iterative gradient while resetting the boundary conditions on each iteration. 

In [ ]:
# Set up a 2-d grid with np.meshgrid: a 4x8 rectangle, with 80 points per value
x_positions, y_positions = np.meshgrid(np.linspace(-4,4,80), np.linspace(-2,2,40), indexing='ij')
position_array = np.array([x_positions,y_positions])
print(position_array.shape)

# Every point in our meshgrid will have a potential (this is our scalar field)
potential_2d = np.zeros(x_positions.shape)

Below, we plot the position points that we are sampling, where the the points sampled were generated from `np.meshgrid`. 

In [ ]:
print('x varies across bottom edge and has shape: ', potential_2d[:,0].shape)
print('x varies across top edge and has shape: ', potential_2d[:,-1].shape)
print('y varies across left edge and has shape: ', potential_2d[0,:].shape)
print('y varies across right edge and has  shape: ', potential_2d[-1,:].shape)

# Visualization of points sampled
plt.scatter(x_positions, y_positions, s=0.2, alpha=0.5)
plt.xlim(-5,5)

# Horizontal lines
plt.axhline(-2, xmin=-5,xmax=5, c='r', ls=':')
plt.axhline(2, xmin=-5,xmax=5, c='r', ls=':')

# Vertical lines
plt.axvline(-4, ymin=-5,ymax=5)
plt.axvline(4, ymin=-5,ymax=5)

# Label what we will want to happen
plt.annotate('V = 100V', (0, 2.1))
plt.annotate('V = -100V', (0, -2.4))

plt.annotate('V=0V', (-4.4,-2.3), rotation=90)
plt.annotate('V=0V', (4.,-2.3), rotation=-90)

plt.ylim(-5,5)
plt.gca().set_aspect('equal')
plt.xlabel('x [m]', fontsize='xx-large')
plt.ylabel('y [m]', fontsize='xx-large')

**Set boundaries exercise** (2 points): Analogous to what we did for a 1-d array, let us do the same thing for a 2-d array, `potential`.  We wish to set boundary conditions such that:
```
x_edges = 0
y_edge_top = 100
y_edge_bottom = -100
```
See the plot we made above for the visualization of the boundary conditions.  Note, you should be able to see that the edge that is shorter has the 0 potential.  We printed out the shape of the array's edges above (we sliced the array to access elements along the edges).  **Note:** There are overlapping corners.  Set the short edges first, and the long edges second so the corners are set to -100 or 100.

There are two ways you can try to do this:

- Identify the edges through the first and/or last element indexing (e.g. 0 or -1) or
- Identify the indices corresponding the the min/max x positions or the min/max y positions (`np.where` would be useful for this.

The latter approach would be agnostic to indexing order.  Use the former (first bullet point) for this exercise.  You will see the second in use in the function `set_boundary_conditions`.

In [ ]:
# Define boundary conditions
x_edges = 0
y_edge_top = 100
y_edge_bottom = -100

# Impose boundary conditions below
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert((potential_2d[:,0]==-100).all())

Below, we write a function that finds the edges of an array based on the position array (using `np.where` statement from the last tutorial/hw).  We then set the corresponding potential array to the specified boundary conditions. 

In [ ]:
def set_boundary_conditions(potential_array, pts_in_meshgrid, 
                            bounds_left=0, bounds_right=0, 
                            bounds_top=100, bounds_bottom=-100) :
    """ Returns potential array with set boundary conditions.  Sets in order of x edges, y edges, then z edges (if 3-d)

    Inputs:
    potential_array (n by m-darray) : array containing the potential at each sampled point 
    pts_in_meshgrid (n by m-darray) : x, y, and z positions for field points
    bounds_right (float) : value of potential on right edge
    bounds_left (float) : value of potential on left edge
    bounds_top (float) : value of potential on top edge
    bounds_bottom (float) : value of potential on bottom edge

    Outputs:
    potential_array (n by m-darray) : array containing the potential at each sampled point with boundary conditions set 
    """
    
    assert(potential_array.shape == pts_in_meshgrid[0].shape)
    
    # Separate out the position arrays along each axis for readability
    x_positions = pts_in_meshgrid[0]
    y_positions = pts_in_meshgrid[1]

    # Identify the edges
    right_edge_mask = np.where(x_positions == x_positions.max())
    left_edge_mask = np.where(x_positions == x_positions.min())
    top_edge_mask = np.where(y_positions == y_positions.max())
    bottom_edge_mask = np.where(y_positions == y_positions.min())
    
    # Set the bounds
    potential_array[right_edge_mask] = bounds_right    
    potential_array[left_edge_mask] = bounds_left
    potential_array[top_edge_mask] = bounds_top
    potential_array[bottom_edge_mask] = bounds_bottom

    return potential_array

**Open question** (2 points): 
- What do the `right_edge_mask` (analogously `left_edge_mask`, etc.) represent?  What does `x_positions[right_edge_mask]` give you?

YOUR ANSWER HERE

**Testing a numpy function in 2d** (2 points) : Define a test array, 
```
test_array = np.arange(15)
test_array = np.reshape(test_array, (3,5))
```

Use `np.roll` to define `test_array_left` and `test_array_right`, where these new arrays respectively correspond to an $i+1$ and $i-1$ element.  *Hint*: In 2d, you'll want to define the key word axis in `np.roll`.

In [ ]:
# Define test_array, test_array_left, and test_array_right
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(test_array.shape == (3,5))
assert(test_array_left.shape == test_array.shape)
assert(((test_array_right - test_array_left)[:,1:-1] < 0).all())

**Do the other directions**: Use `np.roll` to define `test_array_top` and `test_array_bottom`, where these new arrays respectively correspond to an $j-1$ and $j+1$ element.  *Hint*: In 2d, you'll want to define the key word axis in `np.roll`.

In [ ]:
# Define test_array_top and test_array_bottom here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(((test_array_top - test_array_bottom)[1:-1,:] < 0).all())

**The relaxing function** (3 points) : Fill out the function below that relaxes the 2d potential, and uses the function `set_boundary_conditions`.
- You'll want to find the potential in all four directions, e.g. `potential_top`, etc.  See previous exercises for this. 
- You'll want to set the boundary conditions.  Full credit for using the function above, `set_boundary_conditions`.

In [ ]:
def relax_potential_2d( potential_array, pts_in_meshgrid, 
                        bounds_left=0, bounds_right=0, 
                        bounds_top=100, bounds_bottom=-100 ) :
    """Returns relaxed 2-d potential array with set boundary conditions.  

    Inputs:
    potential_array (n-darray) : array containing the potential at each sampled point 
    left_edge_condition (float) : value of potential on left edge
    right_edge_condition (float) : value of potential on right edge

    Outputs:
    relaxed_potential (n-darray) : array containing the relaxed potential at each sampled point with boundary conditions set     
    """
    
    # Define potential_left, right, top, and bottom
    # YOUR CODE HERE
    raise NotImplementedError()
    
    relaxed_potential = np.mean(np.array([potential_left, potential_right, 
                                          potential_top, potential_bottom]),axis=0)
    assert(relaxed_potential.shape == potential_array.shape)
    
    # Set boundary conditions
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return relaxed_potential

In [ ]:
"""Execute to check you're on the right track"""
assert(relax_potential_2d(potential_2d, position_array).shape == potential_2d.shape)

**Perform the iteration** (2 points):  Now, iteratively relax the potential.  You should be able to use *very similar code* to the 1d version used in the tutorial that made use of the `while` loop (refactored code).  Some key differences - you'll need a `potential_2d` to initialize and update, and `relax_potential_2d` takes in two positional arguments (`relax_potential_1d` took only one).    

In [ ]:
# Refactored code here

convergence_criterion = 0.01
largest_iteration_difference = 1 # initialize

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you are on the right track"""
assert(largest_iteration_difference <= convergence_criterion)

**Plot the convergence**:  Plot the largest iteration differences as a function of iteration.

In [ ]:
plt.plot(largest_iteration_differences)
plt.yscale('log')

**Calculate the electric field** (2 points):  Write a function `calc_efield_2d` that uses either `np.roll` (you'd need to do this for both $E_x = -\partial V/\partial x$ and $E_y = -\partial V/\partial y$) or `np.gradient` (you'd also need to do this for two different axes using the key word axis).

In [ ]:
def calc_efield_2d(potential_array) : 
    """Calculate the electric field in one dimension

    Inputs:
    potential_array (nxm array) : array containing the potential at each sampled point 

    Outputs:
    efield_array (2xnxm darray) : array containing the corresponding efield at each sampled point     
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(calc_efield_2d(potential_2d).shape == (2,potential_2d.shape[0], potential_2d.shape[1]))

### Visualizing results

If the previous steps worked out for you, you should be able to execute the cells below and see the potential and efield.

**Visualize the potential**:  Below, we use the imshow method of pyplot to visualize the potential.  You can compare this to the visualization of the problem set-up at the beginning of the homework.  Note, we have to take the transpose because of how imshow orders the values in the potential (effectively, this uses 'xy' indexing).

In [ ]:
# Visualize the potential here
plt.imshow(potential_2d.T, origin='lower', cmap=plt.cm.inferno)
plt.xlabel('x', fontsize='xx-large')
plt.ylabel('y', fontsize='xx-large')
plt.colorbar()

**Visualize the efield and equipotentials**:  Here, we use streamplot to visualize the electric field, and contour to visualize equipotentials.  Note, streamplot assumes 'xy' indexing, and can get clunky if we try to adjust for it.  So, the below figure is rotated.

In [ ]:
efield = calc_efield_2d(potential_2d)

emagnitude = 10*np.log(np.linalg.norm(efield, axis=0)) # We will color code by this

fig, ax = plt.subplots(1, figsize=(8,8))

ax.streamplot(y_positions, x_positions,  
              efield[1], efield[0], 
              color=emagnitude, linewidth=1, cmap=plt.cm.inferno,
              density=2, arrowstyle='->', arrowsize=1.5)

contour1 = ax.contour(y_positions,x_positions,potential_2d, levels=np.arange(-100,100,25),cmap=plt.cm.Reds)
ax.clabel(contour1, fontsize=10, colors='black')  # label the contours

ax.set_aspect('equal')
ax.set_xlabel('y-axis ', fontsize=16)
ax.set_ylabel('x-axis ', fontsize=16)